In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('ftes_lorvec.out', delim_whitespace=True, comment='#', skip_blank_lines=True,
            header=None, names=['energy', 'transition_strength'])

In [6]:
df = df[(df.energy >= 0.1) & (df.energy <= 30)] # MeV

In [8]:
df

,energy,transition_strength
10,0.10,0.135900
11,0.11,0.136607
12,0.12,0.137321
13,0.13,0.138042
14,0.14,0.138770
15,0.15,0.139505
16,0.16,0.140247
17,0.17,0.140996
18,0.18,0.141753
19,0.19,0.142518


In [10]:
df.iloc[::10, :]

,energy,transition_strength
10,0.1,0.135900
20,0.2,0.143290
30,0.3,0.151455
40,0.4,0.160519
50,0.5,0.170631
60,0.6,0.181975
70,0.7,0.194774
80,0.8,0.209311
90,0.9,0.225938
100,1.0,0.245101


In [ ]:
user_dict = {12: {'Category 1': {'att_1': 1, 'att_2': 'whatever'},
                  'Category 2': {'att_1': 23, 'att_2': 'another'}},
             15: {'Category 1': {'att_1': 10, 'att_2': 'foo'},
                  'Category 2': {'att_1': 30, 'att_2': 'bar'}}}

In [11]:
user_dict = {12: {'Category 1': {'att_1': 1, 'att_2': 'whatever'},
                  'Category 2': {'att_1': 23, 'att_2': 'another'}},
             15: {'Category 1': {'att_1': 10, 'att_2': 'foo'},
                  'Category 2': {'att_1': 30, 'att_2': 'bar'}}}

In [60]:
df = pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()},
                       orient='index')

In [61]:
df.index

MultiIndex(levels=[[12, 15], ['Category 1', 'Category 2']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [13]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
         ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [14]:
tuples = list(zip(*arrays))

In [15]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [16]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [17]:
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [20]:
s = pd.Series(np.random.randn(8), index=index)


In [19]:
import numpy as np

In [21]:
s

first  second
bar    one      -0.238791
       two      -0.529749
baz    one       0.309827
       two       0.116655
foo    one      -0.883152
       two      -0.306506
qux    one       1.662731
       two       1.437400
dtype: float64

In [52]:
s.index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [53]:
s.loc['baz']

second
one    0.309827
two    0.116655
dtype: float64

In [22]:
import pandas as pd

In [23]:
stocks = pd.read_csv('http://bit.ly/smallstocks')

In [24]:
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [25]:
stocks.index

RangeIndex(start=0, stop=9, step=1)

In [26]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

In [28]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()

In [29]:
ser.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['Symbol', 'Date'])

In [30]:
ser.unstack()

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [33]:
df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')
df

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [34]:
ser

Symbol  Date      
AAPL    2016-10-03    112.52
        2016-10-04    113.00
        2016-10-05    113.05
CSCO    2016-10-03     31.50
        2016-10-04     31.35
        2016-10-05     31.59
MSFT    2016-10-03     57.42
        2016-10-04     57.24
        2016-10-05     57.64
Name: Close, dtype: float64

In [35]:
ser.loc['AAPL']

Date
2016-10-03    112.52
2016-10-04    113.00
2016-10-05    113.05
Name: Close, dtype: float64

In [36]:
ser.loc['AAPL', '2016-10-03']

112.52

In [37]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

In [38]:
df.loc['AAPL']

Date
2016-10-03    112.52
2016-10-04    113.00
2016-10-05    113.05
Name: AAPL, dtype: float64

In [39]:
df.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: 2016-10-03, dtype: float64

In [41]:
stocks.set_index(['Symbol', 'Date'], inplace=True)

In [42]:
stocks

,,Close,Volume
Symbol,Date,,
CSCO,2016-10-03,31.50,14070500
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500
AAPL,2016-10-04,113.00,29736800
MSFT,2016-10-04,57.24,20085900
CSCO,2016-10-04,31.35,18460400
MSFT,2016-10-05,57.64,16726400
CSCO,2016-10-05,31.59,11808600
AAPL,2016-10-05,113.05,21453100


In [43]:
stocks.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[1, 0, 2, 0, 2, 1, 2, 1, 0], [0, 0, 0, 1, 1, 1, 2, 2, 2]],
           names=['Symbol', 'Date'])

In [44]:
stocks.sort_index(inplace=True)

In [45]:
stocks

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [46]:
stocks.loc['AAPL']

,Close,Volume
Date,,
2016-10-03,112.52,21701800
2016-10-04,113.00,29736800
2016-10-05,113.05,21453100


In [51]:
stocks.loc[('AAPL', '2016-10-03'), :] # rows, columns

Close          112.52
Volume    21701800.00
Name: (AAPL, 2016-10-03), dtype: float64

In [54]:
stocks.loc[('AAPL', '2016-10-03'), 'Close']

112.52

In [55]:
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]

,,Close,Volume
Symbol,Date,,
AAPL,2016-10-03,112.52,21701800
MSFT,2016-10-03,57.42,19189500


In [56]:
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), 'Close']

Symbol  Date      
AAPL    2016-10-03    112.52
MSFT    2016-10-03     57.42
Name: Close, dtype: float64

In [57]:
stocks.loc[('AAPL', ['2016-10-03', '2016-10-04']), 'Close']

Symbol  Date      
AAPL    2016-10-03    112.52
        2016-10-04    113.00
Name: Close, dtype: float64

In [59]:
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']), :]

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900

In [ ]:
both = pd.merge

In [ ]:
 Z=  50 A=  90
  U[MeV]  fE1[mb/MeV]
    0.100   6.291E-03
    0.200   6.776E-03
    0.300   7.300E-03

  Z=  50 A=  91
  U[MeV]  fE1[mb/MeV]
    0.100   6.350E-03
    0.200   6.840E-03
    0.300   7.369E-03
    0.400   7.937E-03

In [ ]:
{50: {90: {}, 91: {}}}

In [83]:
mydict = {50: {90: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]},
               91: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]}},
          51: {90: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]},
               91: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]}}}

In [84]:
{(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()}

{(50, 90): {'U': [0.1, 0.2, 29.9, 30.0],
  'fE1': [0.006291, 0.006776, 0.822, 0.811]},
 (50, 91): {'U': [0.1, 0.2, 29.9, 30.0],
  'fE1': [0.006291, 0.006776, 0.822, 0.811]},
 (51, 90): {'U': [0.1, 0.2, 29.9, 30.0],
  'fE1': [0.006291, 0.006776, 0.822, 0.811]},
 (51, 91): {'U': [0.1, 0.2, 29.9, 30.0],
  'fE1': [0.006291, 0.006776, 0.822, 0.811]}}

In [85]:
df = pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()},
                       orient='index')

In [86]:
df.index

MultiIndex(levels=[[50, 51], [90, 91]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [87]:
df

U                                 fE1
50 90  [0.1, 0.2, 29.9, 30.0]  [0.006291, 0.006776, 0.822, 0.811]
   91  [0.1, 0.2, 29.9, 30.0]  [0.006291, 0.006776, 0.822, 0.811]
51 90  [0.1, 0.2, 29.9, 30.0]  [0.006291, 0.006776, 0.822, 0.811]
   91  [0.1, 0.2, 29.9, 30.0]  [0.006291, 0.006776, 0.822, 0.811]

In [66]:
df = pd.DataFrame.from_dict(user_dict)

In [67]:
df

,50,51
90,"{'U': [0.1, 0.2, 0.3], 'fE1': [0.006291, 0.006...","{'U': [0.1, 0.2, 0.3], 'fE1': [0.006291, 0.006..."
91,"{'U': [0.1, 0.2, 0.3], 'fE1': [0.006291, 0.006...","{'U': [0.1, 0.2, 0.3], 'fE1': [0.006291, 0.006..."


In [68]:
pd.DataFrame.from_dict?

In [113]:
import re

In [114]:
new_line = re.compile('\n[\s\r]+?\n')

In [115]:
contents = open('z050').read()

In [116]:
blocks = new_line.split(contents)

In [117]:
len(blocks)

83

In [208]:
class ConfigurationSyntaxError(Exception):
    pass

In [119]:
blocks[2].splitlines()

[' Z=  50 A=  92',
 '  U[MeV]  fE1[mb/MeV]',
 '    0.100   6.221E-03',
 '    0.200   6.701E-03',
 '    0.300   7.218E-03',
 '    0.400   7.775E-03',
 '    0.500   8.370E-03',
 '    0.600   9.004E-03',
 '    0.700   9.678E-03',
 '    0.800   1.039E-02',
 '    0.900   1.115E-02',
 '    1.000   1.194E-02',
 '    1.100   1.278E-02',
 '    1.200   1.367E-02',
 '    1.300   1.459E-02',
 '    1.400   1.556E-02',
 '    1.500   1.658E-02',
 '    1.600   1.765E-02',
 '    1.700   1.876E-02',
 '    1.800   1.993E-02',
 '    1.900   2.115E-02',
 '    2.000   2.242E-02',
 '    2.100   2.375E-02',
 '    2.200   2.514E-02',
 '    2.300   2.660E-02',
 '    2.400   2.812E-02',
 '    2.500   2.971E-02',
 '    2.600   3.137E-02',
 '    2.700   3.312E-02',
 '    2.800   3.497E-02',
 '    2.900   3.691E-02',
 '    3.000   3.898E-02',
 '    3.100   4.118E-02',
 '    3.200   4.356E-02',
 '    3.300   4.618E-02',
 '    3.400   4.911E-02',
 '    3.500   5.254E-02',
 '    3.600   5.681E-02',
 '    3.700   6.258

In [209]:
raise ConfigurationSyntaxError('blas')

ConfigurationSyntaxError: blas

In [327]:
tst = {'50': {}}

In [ ]:
mydict = {50: {90: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]},
               91: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]}},
          51: {90: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]},
               91: {'U': [0.100, 0.200, 29.900, 30.000], 'fE1': [6.291E-03, 6.776E-03, 8.220E-01, 8.110E-01]}}}

In [328]:
def fn_to_dict(fname):
    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    contents = open(fname).read()
    
    new_line = re.compile('\n[\s\r]+?\n')
    blocks = new_line.split(contents)
    
    assert len(blocks[:-1]) == 82, "Not right number of blocks!"
    assert blocks[-1].strip() == '', "Last line not empty!"
    
    mydict = {str(z_from_fname): {}}
    minA, maxA = 400, 0
    for blk in blocks[:-1]: # except last line
        block = blk.splitlines()
    
        nucleus_header = block[0].split()
        Z, A = nucleus_header[1::2]
        if (nucleus_header[0::2] != ['Z=', 'A=']) or (int(Z) != z_from_fname):
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)
        if int(A) < minA:
            minA = int(A)
        if int(A) > maxA:
            maxA = int(A)
        
        col_header = block[1].split()
        if col_header[:2] != ['U[MeV]', 'fE1[mb/MeV]']:
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)

        columns = (row.split() for row in block[2:])
        c1_vals, c2_vals = zip(*columns)

        d = {A: {'U': c1_vals, 'fE1': c2_vals}}
        mydict[Z].update(d)
        
    print(f"Amax - Amin = {maxA - minA}, {len(blocks[:-1])} blocks")
    return mydict, minA, maxA

In [329]:
md, a, b = fn_to_dict('z050')
print(f"Amin = {a}, Amax = {b}")

Amax - Amin = 88, 82 blocks
Amin = 90, Amax = 178


In [331]:
pprint.pprint(md)

{'50': {'100': {'U': ('0.100',
                      '0.200',
                      '0.300',
                      '0.400',
                      '0.500',
                      '0.600',
                      '0.700',
                      '0.800',
                      '0.900',
                      '1.000',
                      '1.100',
                      '1.200',
                      '1.300',
                      '1.400',
                      '1.500',
                      '1.600',
                      '1.700',
                      '1.800',
                      '1.900',
                      '2.000',
                      '2.100',
                      '2.200',
                      '2.300',
                      '2.400',
                      '2.500',
                      '2.600',
                      '2.700',
                      '2.800',
                      '2.900',
                      '3.000',
                      '3.100',
                      '3.200',
        

                      '14.900',
                      '15.000',
                      '15.100',
                      '15.200',
                      '15.300',
                      '15.400',
                      '15.500',
                      '15.600',
                      '15.700',
                      '15.800',
                      '15.900',
                      '16.000',
                      '16.100',
                      '16.200',
                      '16.300',
                      '16.400',
                      '16.500',
                      '16.600',
                      '16.700',
                      '16.800',
                      '16.900',
                      '17.000',
                      '17.100',
                      '17.200',
                      '17.300',
                      '17.400',
                      '17.500',
                      '17.600',
                      '17.700',
                      '17.800',
                      '17.900',
        

                      '11.400',
                      '11.500',
                      '11.600',
                      '11.700',
                      '11.800',
                      '11.900',
                      '12.000',
                      '12.100',
                      '12.200',
                      '12.300',
                      '12.400',
                      '12.500',
                      '12.600',
                      '12.700',
                      '12.800',
                      '12.900',
                      '13.000',
                      '13.100',
                      '13.200',
                      '13.300',
                      '13.400',
                      '13.500',
                      '13.600',
                      '13.700',
                      '13.800',
                      '13.900',
                      '14.000',
                      '14.100',
                      '14.200',
                      '14.300',
                      '14.400',
        

                      '7.800',
                      '7.900',
                      '8.000',
                      '8.100',
                      '8.200',
                      '8.300',
                      '8.400',
                      '8.500',
                      '8.600',
                      '8.700',
                      '8.800',
                      '8.900',
                      '9.000',
                      '9.100',
                      '9.200',
                      '9.300',
                      '9.400',
                      '9.500',
                      '9.600',
                      '9.700',
                      '9.800',
                      '9.900',
                      '10.000',
                      '10.100',
                      '10.200',
                      '10.300',
                      '10.400',
                      '10.500',
                      '10.600',
                      '10.700',
                      '10.800',
                      '10.900'

                        '2.355E+00',
                        '2.312E+00',
                        '2.269E+00',
                        '2.229E+00',
                        '2.189E+00',
                        '2.151E+00',
                        '2.114E+00',
                        '2.078E+00',
                        '2.044E+00',
                        '2.011E+00',
                        '1.978E+00',
                        '1.947E+00',
                        '1.917E+00',
                        '1.888E+00',
                        '1.860E+00',
                        '1.832E+00',
                        '1.806E+00',
                        '1.781E+00',
                        '1.756E+00',
                        '1.732E+00',
                        '1.709E+00',
                        '1.687E+00',
                        '1.665E+00',
                        '1.644E+00',
                        '1.623E+00',
                        '1.604E+00',
                        '1.584E+00',
 

...

                      '8.000',
                      '8.100',
                      '8.200',
                      '8.300',
                      '8.400',
                      '8.500',
                      '8.600',
                      '8.700',
                      '8.800',
                      '8.900',
                      '9.000',
                      '9.100',
                      '9.200',
                      '9.300',
                      '9.400',
                      '9.500',
                      '9.600',
                      '9.700',
                      '9.800',
                      '9.900',
                      '10.000',
                      '10.100',
                      '10.200',
                      '10.300',
                      '10.400',
                      '10.500',
                      '10.600',
                      '10.700',
                      '10.800',
                      '10.900',
                      '11.000',
                      '11.10

                        '1.058E+01',
                        '1.087E+01',
                        '1.105E+01',
                        '1.110E+01',
                        '1.104E+01',
                        '1.088E+01',
                        '1.063E+01',
                        '1.029E+01',
                        '9.902E+00',
                        '9.481E+00',
                        '9.061E+00',
                        '8.673E+00',
                        '8.339E+00',
                        '8.076E+00',
                        '7.887E+00',
                        '7.771E+00',
                        '7.725E+00',
                        '7.749E+00',
                        '7.849E+00',
                        '8.031E+00',
                        '8.300E+00',
                        '8.659E+00',
                        '9.107E+00',
                        '9.637E+00',
                        '1.023E+01',
                        '1.087E+01',
                        '1.152E+01',
 

                        '3.967E+00',
                        '3.885E+00',
                        '3.806E+00',
                        '3.731E+00',
                        '3.659E+00',
                        '3.590E+00',
                        '3.523E+00',
                        '3.460E+00',
                        '3.399E+00',
                        '3.340E+00',
                        '3.284E+00',
                        '3.229E+00',
                        '3.177E+00',
                        '3.127E+00',
                        '3.078E+00',
                        '3.031E+00',
                        '2.986E+00',
                        '2.942E+00',
                        '2.899E+00',
                        '2.857E+00',
                        '2.816E+00',
                        '2.776E+00',
                        '2.737E+00',
                        '2.699E+00',
                        '2.661E+00',
                        '2.625E+00',
                        '2.588E+00',
 

                       '2.956E+00',
                       '2.869E+00',
                       '2.785E+00',
                       '2.705E+00',
                       '2.629E+00',
                       '2.555E+00',
                       '2.485E+00',
                       '2.418E+00',
                       '2.353E+00',
                       '2.292E+00',
                       '2.233E+00',
                       '2.177E+00',
                       '2.123E+00',
                       '2.072E+00',
                       '2.024E+00',
                       '1.977E+00',
                       '1.933E+00',
                       '1.891E+00',
                       '1.851E+00',
                       '1.813E+00',
                       '1.777E+00',
                       '1.743E+00',
                       '1.710E+00',
                       '1.679E+00',
                       '1.649E+00',
                       '1.621E+00',
                       '1.594E+00',
                       '1.56

                       '3.686E-02',
                       '3.887E-02',
                       '4.101E-02',
                       '4.330E-02',
                       '4.579E-02',
                       '4.856E-02',
                       '5.173E-02',
                       '5.558E-02',
                       '6.063E-02',
                       '6.752E-02',
                       '7.525E-02',
                       '8.161E-02',
                       '8.674E-02',
                       '9.126E-02',
                       '9.551E-02',
                       '9.966E-02',
                       '1.038E-01',
                       '1.080E-01',
                       '1.122E-01',
                       '1.166E-01',
                       '1.211E-01',
                       '1.257E-01',
                       '1.304E-01',
                       '1.353E-01',
                       '1.404E-01',
                       '1.456E-01',
                       '1.510E-01',
                       '1.56

In [306]:
b

178

In [307]:
b - a

88

In [312]:
df = pd.DataFrame.from_dict({(i,j): md[i][j][v] for i in md.keys() for j in md[i].keys() for v in md[i][j]}, orient='index')

In [ ]:
mydict = {(i,j): md[i][j][v] for i in md.keys() for j in md[i].keys() for v in md[i][j]}

In [251]:
import pprint

In [252]:
pprint.pprint(mydict)

{'50': {'178': {'U': ('0.100',
                      '0.200',
                      '0.300',
                      '0.400',
                      '0.500',
                      '0.600',
                      '0.700',
                      '0.800',
                      '0.900',
                      '1.000',
                      '1.100',
                      '1.200',
                      '1.300',
                      '1.400',
                      '1.500',
                      '1.600',
                      '1.700',
                      '1.800',
                      '1.900',
                      '2.000',
                      '2.100',
                      '2.200',
                      '2.300',
                      '2.400',
                      '2.500',
                      '2.600',
                      '2.700',
                      '2.800',
                      '2.900',
                      '3.000',
                      '3.100',
                      '3.200',
        

In [153]:
Z, A = block[0].split()[1], block[0].split()[3]
print(f"(Z, A) = ({Z}, {A})")

(Z, A) = (50, 100)


In [154]:
col1, col2 = block[1].split()[0].split('[')[0], block[1].split()[1].split('[')[0]
print(f"(col1, col2) = ({col1}, {col2})")

(col1, col2) = (U, fE1)


In [ ]:
myd = {(i,j): md[i][j][v] for i in md.keys() for j in md[i].keys() for v in md[i][j]}

In [234]:
df = pd.DataFrame.from_dict({(i,j): myd[i][j] for i in myd.keys() for j in myd[i].keys()}, orient='index')

In [235]:
df

,,U,fE1
50,100,"(0.100, 0.200, 0.300, 0.400, 0.500, 0.600, 0.7...","(6.263E-03, 6.746E-03, 7.266E-03, 7.825E-03, 8..."


In [313]:
print(df.to_string())

                 0          1          2          3          4          5          6          7          8          9          10         11         12         13         14         15         16         17         18         19         20         21         22         23         24         25         26         27         28         29         30         31         32         33         34         35         36         37         38         39         40         41         42         43         44         45         46         47         48         49         50         51         52         53         54         55         56         57         58         59         60         61         62         63         64         65         66         67         68         69         70         71         72         73         74         75         76         77         78         79         80         81         82         83         84         85         86         87         88         89  

In [314]:
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
"(50, 178)",6.031E-02,6.518E-02,7.060E-02,7.665E-02,8.338E-02,9.086E-02,9.919E-02,1.085E-01,1.189E-01,1.306E-01,...,1.285E+00,1.275E+00,1.265E+00,1.255E+00,1.246E+00,1.236E+00,1.227E+00,1.217E+00,1.208E+00,1.198E+00


In [332]:
md = {'50': {'100': {'col1': ('0.100',
                      '0.200',
                      '0.300',
                      '0.400'),
                'col2': ('6.263E-03',
                        '6.746E-03',
                        '7.266E-03',
                        '7.825E-03')},
        '101': {'col1': ('0.100',
                      '0.200',
                      '0.300',
                      '0.400'),
                'col2': ('6.510E-03',
                        '7.011E-03',
                        '7.553E-03',
                        '8.134E-03')}
             }
}

In [334]:
md1 = {(i,j): md[i][j][v] for i in md.keys() for j in md[i].keys() for v in md[i][j]}